# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [1]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

NameError: name 'load_dotenv' is not defined

# Import Data

In [ ]:
# Import the necessary CSVs to Pandas DataFrames
# Read the census data into a Pandas DataFrame
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

# checking for NA values 
number_of_na = sfo_data.isna().sum().sum()
print(f"Number of NA values in the SFO Census data frame: {number_of_na}")


# Checking for Null values in data 
number_of_nulls = sfo_data.isnull().sum().sum()
print(f"Number of null values in the SFO Census data frame: {number_of_nulls}")

# Dropping the null values from the data frame
sfo_data.dropna(inplace = True)

# Checking for Null values in data 
number_of_nulls = sfo_data.isnull().sum().sum()
print(f"After cleaning number of null values in the SFO Census data frame: {number_of_nulls}")

# checking for NA values 
number_of_na = sfo_data.isna().sum().sum()
print(f"After cleaning number of NA values in the SFO Census data frame: {number_of_na}")

# Load neighborhoods coordinates data
# Read the cordinate data into a Pandas DataFrame
file_path = Path("Data/neighborhoods_coordinates.csv")

coordinate_data_df = pd.read_csv(file_path)
coordinate_data_df.columns = ["neighborhood", "lat", "lon"] 
coordinate_data_df = coordinate_data_df.set_index("neighborhood") 

# checking for NA values 
number_of_na_coordinate_data = coordinate_data_df.isna().sum().sum()
print(f"Number of NA values in the Neighborhood Cordinates data frame: {number_of_na_coordinate_data}")

# Checking for Null values in data 
number_of_nulls_coordinate_data = coordinate_data_df.isnull().sum().sum()
print(f"Number of null values in the Neighborhood Cordinates data frame: {number_of_nulls_coordinate_data}")

# Dropping the null values from the data frame
coordinate_data_df.dropna(inplace = True)

# Checking for Null values in data 
number_of_nulls_coordinate_data = coordinate_data_df.isnull().sum().sum()
print(f"After cleaning number of null values in the Neighborhood Cordinates data frame: {number_of_nulls_coordinate_data}")

# checking for NA values 
number_of_na_coordinate_data = coordinate_data_df.isna().sum().sum()
print(f"After cleaning number of NA values in the Neighborhood Cordinates data frame: {number_of_na_coordinate_data}")

# Global DataFrames (used for multiple graphs)


In [13]:
""""" Global DataFrames were created to improve code preformance """"

# Group by year and neighborhood and then create a new dataframe of all the means 
means = sfo_data.groupby(["year", "neighborhood"]).mean()
means_df = pd.DataFrame(means)

# Creating a data frame sales price per neighborhood, housing units, gross rent for each neighborhood averaged through the years 
mean_values_neighborhood = sfo_data.groupby(["neighborhood"]).mean()
mean_values_neighborhood_df = pd.DataFrame(mean_values_neighborhood) 

# Filter out 10 most expensive neighborhoods interms of sales price per square foot
top_10_mean_sale_price_of_neighborhood_df = mean_values_neighborhood_df.nlargest(10, "sale_price_sqr_foot")

# Restting index for the top 10 neighborhoods dataframe
top_10_mean_sale_price_of_neighborhood_no_index_df = top_10_mean_sale_price_of_neighborhood_df.reset_index()

# Creating a list containing the names of the top 10 neighborhoods 
top_ten_neighborhoods = top_10_mean_sale_price_of_neighborhood_no_index_df["neighborhood"].tolist()

# Resetting and creating a dataframe from the pricipal data frame without index
sfo_data_no_index = sfo_data.reset_index()

# Selecting a subset containg the top 10 neighborhood names from the principal sfo_data_no_index dataframe
yearly_change_of_rent_and_price_top_10_no_index = sfo_data_no_index[sfo_data_no_index["neighborhood"].isin(top_ten_neighborhoods)]  

# Setting the index of the top 10 neighborhood data with yearly variaion included to neighborhoods 
yearly_change_of_rent_and_price_top_10 = yearly_change_of_rent_and_price_top_10_no_index.set_index(["neighborhood"])

# Using previously calculated mean_values_neighborhood_df data frame for the mean values for each neighborhood
# Join the average values with the neighborhood locations
combined_sfo_df = pd.concat([mean_values_neighborhood_df, coordinate_data_df], axis="columns", join="inner")
combined_sfo_df.head()

# Removing the index of the combined 
combined_sfo_no_index_df = combined_sfo_df.reset_index()

# Checking the data structure
combined_sfo_no_index_df.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent,lat,lon
0,Alamo Square,366.020712,378401.0,2817.285714,37.791012,-122.402100
1,Anza Vista,373.382198,379050.0,3031.833333,37.779598,-122.443451
2,Bayview,204.588623,376454.0,2318.400000,37.734670,-122.401060
3,Bayview Heights,590.792839,382295.0,3739.000000,37.728740,-122.410980
4,Buena Vista Park,452.680591,378076.5,2698.833333,37.768160,-122.439330


## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [14]:
# Define Panel Visualization Functions

def housing_units_per_year_optional(): # Optional function show as this is similar to the other but scales the y axis taking account the min, max, and the std
    """Housing Units Per Year."""
    # Calculate the mean number of housing units per year 
    average_housing_units_per_year_optional = sfo_data["housing_units"].groupby("year").mean()
    average_housing_units_per_year_optional_df = pd.DataFrame(average_housing_units_per_year_optional) 
    
    # Optional Challenge: Use the min, max, and std to scale the y limits of the chart
    # Calculating necessary statistical parameters: min, max, and std
    min_of_annual_unit_averages = int(average_housing_units_per_year_optional_df.min())
    max_of_annual_unit_averages = int(average_housing_units_per_year_optional_df.max())
    std_of_annual_unit_averages = int(average_housing_units_per_year_optional_df.std())

    # Calculating the y-axis upper limit = max_height_annual_unit_averages and y-axis lower limit = min_height_annual_unit_averages and coverting to integer format  
    max_height_annual_unit_averages = max_of_annual_unit_averages + std_of_annual_unit_averages
    min_height_annual_unit_averages = min_of_annual_unit_averages - std_of_annual_unit_averages

    #plotting the bar graph with the calculated y-axis range
    housing_units_per_year_plot_optional = average_housing_units_per_year_optional_df.hvplot.bar(
        height = 400, 
        width = 800,
        x = "year", 
        y = "housing_units",
        xlabel = "Year", 
        ylabel = "Average Housing Units", 
        ylim = (min_height_annual_unit_averages, max_height_annual_unit_averages), 
        title = "Average Annual Housing Unit Occupation in San Francisco Optional\n"
    )
    return housing_units_per_year_plot_optional


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    # Calculate the average gross rent
    average_monthly_rent = sfo_data["gross_rent"].groupby(["year"]).mean()
    average_monthly_rent_df = pd.DataFrame(average_monthly_rent) 
    
    # Line chart for average montly rent
    # Manually adjust the y limit of the chart using the min and max values
    average_gross_rent_plot = average_monthly_rent_df.hvplot.line(
        height = 400, 
        width = 800,
        x = "year", 
        y = "gross_rent",
        xlabel = "Year", 
        line_color = "Red", 
        line_width = 2.5, 
        ylabel = "Average Monthley Rent", 
        ylim = (800, 5000), 
        title = "Yearly Variation of the Average Rent in San Francisco\n"
    )
    return average_gross_rent_plot


def average_sales_price():
    """Average Sales Price Per Year."""
    # Calculate the average sale price per square foot
    average_sales_price_per_square_foot = sfo_data["sale_price_sqr_foot"].groupby(["year"]).mean()
    average_sales_price_per_square_foot_df = pd.DataFrame(average_sales_price_per_square_foot) 
    
    # Line chart for average sales per square foot
    # Manually adjust the y limit of the chart using the min and max values
    average_sales_price_plot = average_sales_price_per_square_foot_df.hvplot.line(
        height = 400, 
        width = 800,
        x = "year", 
        y = "sale_price_sqr_foot", 
        xlabel = "Year", 
        line_color = "Green", 
        line_width = 2.5, 
        ylabel = "Average Sales Price/Square Foot", 
        ylim = (300, 800), 
        title = "Yearly Variation of the Average Sale Price/Sqr Foot in San Francisco\n"
    )  
    return average_sales_price_plot


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # Use hvplot to create an interactive line chart of the average price per sq ft.
    # The plot should have a dropdown selector for the neighborhood
    average_price_by_neighborhood_plot = means_df.hvplot.line(
        height = 400, 
        width = 800,
        x = "year", 
        y = "sale_price_sqr_foot", 
        line_color = "Red", 
        xlabel = "Year" , 
        ylabel = "Sales Price per Square Foot", 
        ylim = (0, 1200),    
        groupby = "neighborhood",
        title = "Yearly Sale Price/Sqr Foot Variation for Each Neighborhood"
    )
    return average_price_by_neighborhood_plot
    
       
def rent_by_neighborhood():
    """Average Prices by Neighborhood."""

   # Use hvplot to create an interactive line chart of the average monthly rent.
    # The plot should have a dropdown selector for the neighborhood
    rent_by_neighborhood_plot = means_df.hvplot.line(
        height = 400, 
        width = 800,
        x = "year", 
        y = "gross_rent", 
        line_color = "Orange",
        xlabel = "Year", 
        ylabel = "Gross Rent", 
        ylim = (0, 5000),    
        groupby = "neighborhood",
        title = "Yearly Gross Rent Variation for each Neighborhood"
    )
    return rent_by_neighborhood_plot


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    # Plotting the data from the top 10 expensive neighborhoods
    top_most_expensive_neighborhoods_plot = top_10_mean_sale_price_of_neighborhood_df.hvplot.bar(
        height = 400, 
        width = 800, 
        x = "neighborhood", 
        y = "sale_price_sqr_foot",  
        color = "Purple", 
        xlabel = "Neighborhood", 
        ylabel = "Price per Square Foot" , 
        ylim = (0, 1000), 
        rot = 90, 
        title = "Top Ten Most Expensive Neighborhoods by Price/Sqr Foot"
    )
    return top_most_expensive_neighborhoods_plot


def annual_variation_of_sales_price_and_rent():
    """Comparison of Rent and Sales Prices of All Neighborhoods."""   
  
    # Fetch the previously generated DataFrame that was grouped by year and neighborhood
    # The plot should have a dropdown selector for the neighborhood
    annual_variation_of_sales_price_and_rent_plot = means_df.hvplot.line(
        height = 400, 
        width = 800, 
        y = ["sale_price_sqr_foot", "gross_rent"], 
        x = "year", 
        xlabel = "Year", 
        ylabel = "Cost", 
        ylim = (0, 5000),
        groupby = "neighborhood", 
        rot = 90, 
        title = "Yearly Variation of Sales Price/Sqr Foot vs. Rent for Each Neighborhood"
    )
    return annual_variation_of_sales_price_and_rent_plot



def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    # Fetch the previously generated DataFrame that was grouped by year and neighborhood
    # The plot should have a dropdown selector for the neighborhood
    most_expensive_neighborhoods_rent_sales_plot = yearly_change_of_rent_and_price_top_10.hvplot.bar(
        height = 400, 
        width = 800, 
        x = "year", 
        y = ["sale_price_sqr_foot", "gross_rent"], 
        xlabel = "Year", 
        ylabel = "Cost",
        ylim = (0, 5000),
        groupby = "neighborhood", 
        rot = 90, 
        title = "Yearly Variation of Sales Price/Sqr Foot vs. Rent for top 10 Neighborhoods"
    )
    
    return most_expensive_neighborhoods_rent_sales_plot
    
def most_expensive_neighborhoods_rent_sales_averaged_over_years():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
 
    # Fetch the previously generated DataFrame that was grouped by year and neighborhood
    # The plot should have a dropdown selector for the neighborhood for Top 10 Neighborhoods by sale price per square foot 
    most_expensive_neighborhoods_rent_sales_averaged_over_years_plot = top_10_mean_sale_price_of_neighborhood_df.hvplot.bar(
        height = 400, 
        width = 800, 
        x = "neighborhood", 
        y = ["sale_price_sqr_foot", "gross_rent"], 
        xlabel = "Neighborhood" , 
        ylabel = "Rent or Price Per Square Foot",
        ylim = (0, 5000),
        groupby = "neighborhood", 
        title = "Sale Price/Square Foot & Rent, Averaged over All the Years - Top 10 Neighborhoods"
    )
    
    return most_expensive_neighborhoods_rent_sales_averaged_over_years_plot

def top_10_neighborhood_map():
    """Neighborhood Map."""

    # Set the mapbox access token
    # Set the Mapbox API
    px.set_mapbox_access_token(map_box_api)

    #yearly_change_of_rent_and_price_top_10_no_index = sfo_data_no_index[sfo_data_no_index["neighborhood"].isin(top_ten_neighborhoods)]  
    filtered_data = combined_sfo_no_index_df[combined_sfo_no_index_df["neighborhood"].isin(top_ten_neighborhoods)]

    # Plotting the overlay map of the Top 10 Sales Price/Sqr Foot Neighborhoods
    top_10_neighborhood_map_plot = px.scatter_mapbox(
    filtered_data,
    lat ="lat",
    lon ="lon",
    size ="sale_price_sqr_foot",
    color ="gross_rent",
    hover_name = "neighborhood",
    zoom = 10,
    title = "The Top 10 Most Expensive Neighborhoods",    
    height = 400,
    width = 800    
    )
    return top_10_neighborhood_map_plot


def all_neighborhood_map():
    """Neighborhood Map."""

    # Set the mapbox access token
    # Set the Mapbox API
    px.set_mapbox_access_token(map_box_api)

    #yearly_change_of_rent_and_price_top_10_no_index = sfo_data_no_index[sfo_data_no_index["neighborhood"].isin(top_ten_neighborhoods)]  
    filtered_data = combined_sfo_no_index_df

    # Plotting the overlay map of the All Neighborhoods
    all_neighborhood_map_plot = px.scatter_mapbox(
    filtered_data,
    lat ="lat",
    lon ="lon",
    size ="sale_price_sqr_foot",
    color ="gross_rent",
    hover_name = "neighborhood",
    zoom = 10,
    title = "All San Francisco Neighborhoods",
    height = 400,
    width = 800    
    )
    return all_neighborhood_map_plot

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [15]:
# Dashboard title

# Graphs arranged in a column for tab 1
occupation_rent_column = pn.Column(
    housing_units_per_year_optional(),#Housing unit occupation OPTIONAL
    average_gross_rent(),
    average_sales_price()
)

# Graphs arranged in a column for tab 2
neighborhood_dependent_price_rent_column = pn.Column(
    average_price_by_neighborhood, 
    rent_by_neighborhood, 
    top_most_expensive_neighborhoods
)

# Graphs arranged in a column for tab 3
annual_variations_in_neighborhoods_column = pn.Column(
    annual_variation_of_sales_price_and_rent,  
    most_expensive_neighborhoods_rent_sales, 
    most_expensive_neighborhoods_rent_sales_averaged_over_years
)

# Graphs arranged in a column for tab 4
top_10_neighborhood_column = pn.Column(
    top_10_neighborhood_map,
    all_neighborhood_map
)

# Create the dashboard for SFO housing market analysis
SFO_housing_dashboard = pn.Tabs(
    ("Number of Housing Units & Rent", occupation_rent_column), # Housing unit occupation
    ("Price/Sqr-Foot & Rent per Neighborhood", neighborhood_dependent_price_rent_column),
    ("Annual Variations: Price/Sqr-Foot & Rent per Neighborhood", annual_variations_in_neighborhoods_column),
    ("Neighborhoods on Map", top_10_neighborhood_column)
)

In [16]:
# Show dashboard
SFO_housing_dashboard.servable()

Tabs
    [0] Column
        [0] HoloViews(Bars)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [1] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive07330')
                    [0] HoloViews(DynamicMap, name='interactive07330')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
        [1] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive07367')
                    [0] HoloViews(DynamicMap, name='interactive07367')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
        [2] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive07404')
    [2] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive07489')
                    [0] HoloViews(DynamicMap, name='interactive07489')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
        [1] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive07526')
                    [0] HoloViews(DynamicMap, name='interactive07526')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Cow Hollow', ...], value='Cow Hollow', width=250)
                        [1] VSpacer()
        [2] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive07564')
                    [0] HoloViews(DynamicMap, name='interactive07564')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Union Square District', ...], value='Union Square District', width=250)
                        [1] VSpacer()
    [3] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive07602')
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive07608')

In [17]:
# View as the combined dashboard on a web page
SFO_housing_dashboard.servable().show()

Launching server at http://localhost:65026


# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()